In [2]:
library(purrr)
DF = data.frame(x1 = c(1, 1, 0, 1, 0, 0, 1, 0, 1, 0), 
                x2 = c(0, 1, 0, 1, 0, 1, 1, 0, 1, 0), 
                x3 = c(0, 1, 1, 1, 0, 1, 1, 0, 1, 0))
# DF = as.data.table(DF)
DF

x1,x2,x3
<dbl>,<dbl>,<dbl>
1,0,0
1,1,1
0,0,1
1,1,1
0,0,0
0,1,1
1,1,1
0,0,0
1,1,1


In [26]:
# calcualtes function (12) in the article
g_i_pi = function(i, parents = NA, df) { # i is the place of the variable in the assigned order, not the order of the variable in the database
    R = unique(df[, i])
    r = length(R)

    if (all(is.na(parents))) {
        q = 1
        N = as.data.frame(rbind(table(df[, i])))
    } else {
        W = as.data.frame(cbind(unique(df[, parents])))
        q = nrow(W)
        N = data.frame(matrix(nrow = q, ncol = r))

        for (j in 1:q) {
            df_parents = df[apply(as.data.frame(cbind(df[,parents])), 1, function(row) all(row == as.numeric(W[j,]))),]
            for (k in 1:r) {
                N[j, k] = sum(df_parents[, i] == R[k])
            }
        }
    }

    foo = data.frame(prod = apply(as.data.frame(map(N, factorial)), 1, prod), 
                    sum = apply(N, 1, sum))
    out = prod(unlist(map2(.x = foo$prod, .y = foo$sum, .f = ~ (factorial(r - 1) / factorial(.y + r - 1)) * .x)))
    return(out)
}

In [27]:
# check if the calculation is correct
g_i_pi(i = 2, parents = 1, df = DF)

[1] 0.001111111

In [29]:
# implements K2 algorithm
K2_algorithm = function(n, u, D) {
    parents = vector('list', length = n)
    for (i in 1:n) {
        if (i == 1) {next}

        parents[[i]] = NA 
        len_parents = 0
        P_old = g_i_pi(i, parents[[i]], D)
        # cat('pold:', P_old, '\n')

        OTP = TRUE
        while(OTP & len_parents < u) {
            # cat('\nparents:', parents[[i]], 'variable:', i, '\n')
            # cat('condition:', is.na(parents[[i]]), 'variable:', i, '\n')
            if (is.na(parents[[i]])) {
                indexes = 1:(i-1)
            } else {
                foo = 1:(i-1)
                indexes = foo[-parents[[i]]]
            }
            # cat('indexes:', indexes, 'variable:', i, '\n')

            if (length(indexes) == 0) {
                OTP = FALSE
                next
            }
            P = vector('numeric', length = length(indexes))
            for (t in 1:length(indexes)) {
                cand_parents = append(parents[[i]], indexes[t])
                cand_parents = cand_parents[!is.na(cand_parents)]
                P[t] = g_i_pi(i, cand_parents, D)
            }
            # cat('maxP:', max(P), 'Pold:', P_old, 'i:', i, 'parents:', parents[[i]], '\n')
            if (max(P) > P_old) {
                P_old = max(P)
                parents[[i]] = append(parents[[i]], indexes[which.max(P)])
                parents[[i]] = parents[[i]][!is.na(parents[[i]])]
                len_parents = length(parents[[i]])
            } else {
                OTP = False
            }
        }
    }
    return(parents)
}

In [56]:
# the result is correct 
trial = K2_algorithm(n = 3, u = 1, D = DF)
trial

# indeed we see that var 1 in our order has zero parents
# while var 2 in our order has var 1 as parent
# and finally var 3 has var 2 as parents

[[1]]
NULL

[[2]]
[1] 1

[[3]]
[1] 2

In [57]:
# still have to figure out why this call raises an error
trial = K2_algorithm(n = 3, u = 2, D = DF)
trial

ERROR: Error in K2_algorithm(n = 3, u = 2, D = DF): object 'False' not found
